In [1]:
import pandas as pd
from datetime import datetime
from datetime import date
import json
import math
import xlrd
import numpy as np

pd.set_option('display.float_format','{:.4f}'.format)

hubs_dict = {
    'ARICA': 'IQUIQUE',
    'IQUIQUE': 'IQUIQUE',
    'ANTOFAGASTA': 'ANTOFAGASTA',
    'COPIAPO': 'COPIAPO',
    'COQUIMBO': 'COQUIMBO',
    'OVALLE': 'COQUIMBO',
    'ILLAPEL': 'CURAUMA',
    'LLAY LLAY': 'CURAUMA',
    'CURAUMA': 'CURAUMA',
    'SANTIAGO SUR': 'SANTIAGO SUR',
    'RANCAGUA': 'SANTIAGO SUR',
    'TALCA': 'TALCA',
    'CHILLAN': 'TALCAHUANO',
    'TALCAHUANO': 'TALCAHUANO',
    'LOS ANGELES': 'TALCAHUANO',
    'TEMUCO': 'TEMUCO',
    'VALDIVIA': 'VALDIVIA',
    'OSORNO': 'VALDIVIA',
    'PUERTO MONTT': 'PUERTO MONTT',
    'CASTRO': 'PUERTO MONTT',
    'COYHAIQUE': 'COYHAIQUE',
    'CALAMA': 'ANTOFAGASTA'
}

def stats(datos, confianza):
    
    lead_time = 7

    mean_no_group = datos.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST']).mean().reset_index().rename(columns={'Venta en pallets': 'MEDIA'})
    stdev_no_group = datos.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST']).std().reset_index().rename(columns={'Venta en pallets': 'STD'})

    mean_no_group['MEDIA'] = mean_no_group.apply(lambda x: x.MEDIA * lead_time, axis=1)
    stdev_no_group['STD'] = stdev_no_group.apply(lambda x: x.STD * lead_time, axis=1)

    data_completa = mean_no_group.merge(stdev_no_group, on=['ID_SKU_VENTA', 'DESCR_CENDIST'])

    data_completa['COEF_VAR'] = data_completa.apply(lambda x: x['STD'] / x['MEDIA'] if x['MEDIA'] != 0 else 0, axis=1)

    data_completa['SS'] = data_completa.apply(lambda x: confianza * x['STD'], axis=1)

    data_completa['STOCK_TEORICO'] = data_completa.apply(lambda x: x['MEDIA'] + x['SS'], axis=1)

    data_completa['PORCENTAJE_SS_DEL_TOTAL'] = data_completa.apply(lambda x: x['SS'] / x['STOCK_TEORICO'] if x['MEDIA'] != 0 else 0, axis=1)

    porcentaje_ss_del_total = data_completa['SS'].sum() / data_completa['STOCK_TEORICO'].sum()
    
    return data_completa, porcentaje_ss_del_total

restricted = True
porcentaje = 100


In [2]:
datos_limpios = pd.read_csv('../data/datos_limpios_filtrados.csv')
datos_limpios = datos_limpios[(datos_limpios['DESCR_CENDIST'] != 'CERVECERA') & (datos_limpios['DESCR_CENDIST'] != 'MODELO')]
datos_limpios = datos_limpios.drop(columns='Unnamed: 0')

In [3]:
stats_no_agrupados, porcentaje_ss_no_agrupado = stats(datos_limpios, 1.65)

In [4]:
stats_no_agrupados2, porcentaje_ss_no_agrupado2 = stats(datos_limpios, 1.28)

In [5]:
stats_no_agrupados

,ID_SKU_VENTA,DESCR_CENDIST,MEDIA,STD,COEF_VAR,SS,STOCK_TEORICO,PORCENTAJE_SS_DEL_TOTAL
0,515,ANTOFAGASTA,0.4005,0.7071,1.7653,1.1667,1.5672,0.7444
1,515,ARICA,0.0419,0.1407,3.3622,0.2322,0.2741,0.8473
2,515,CALAMA,0.2196,0.4439,2.0217,0.7325,0.9521,0.7694
3,515,CASTRO,0.1207,0.3007,2.4909,0.4962,0.6169,0.8043
4,515,CHILLAN,0.2766,0.5116,1.8497,0.8442,1.1208,0.7532
...,...,...,...,...,...,...,...,...
23007,875225,VALDIVIA,0.0003,0.0041,14.6616,0.0068,0.0071,0.9603
23008,875359,SANTIAGO SUR,0.0084,0.2427,28.8444,0.4004,0.4088,0.9794
23009,875365,CURAUMA,0.0168,0.3430,20.3838,0.5659,0.5828,0.9711
23010,875365,SANTIAGO SUR,0.0084,0.2427,28.8444,0.4004,0.4088,0.9794


In [6]:
stats_no_agrupados2

,ID_SKU_VENTA,DESCR_CENDIST,MEDIA,STD,COEF_VAR,SS,STOCK_TEORICO,PORCENTAJE_SS_DEL_TOTAL
0,515,ANTOFAGASTA,0.4005,0.7071,1.7653,0.9050,1.3056,0.6932
1,515,ARICA,0.0419,0.1407,3.3622,0.1801,0.2220,0.8114
2,515,CALAMA,0.2196,0.4439,2.0217,0.5683,0.7878,0.7213
3,515,CASTRO,0.1207,0.3007,2.4909,0.3849,0.5057,0.7612
4,515,CHILLAN,0.2766,0.5116,1.8497,0.6549,0.9315,0.7031
...,...,...,...,...,...,...,...,...
23007,875225,VALDIVIA,0.0003,0.0041,14.6616,0.0053,0.0055,0.9494
23008,875359,SANTIAGO SUR,0.0084,0.2427,28.8444,0.3106,0.3190,0.9736
23009,875365,CURAUMA,0.0168,0.3430,20.3838,0.4390,0.4559,0.9631
23010,875365,SANTIAGO SUR,0.0084,0.2427,28.8444,0.3106,0.3190,0.9736


In [8]:
stats_no_agrupados['SS'].sum() - stats_no_agrupados2['SS'].sum()

10602.05032845214

In [15]:
if restricted:    
    FILE_AGRUPACIONES = f'../data/agrupacion_sku_{porcentaje}%.xlsx'
    sheets_dict = pd.read_excel(FILE_AGRUPACIONES, sheet_name=None)

    master = pd.DataFrame()
    for name in sheets_dict:

        df = sheets_dict[name]
        master = pd.concat([master, df])

    dict_sku_cd = {}
    master = master.rename(columns={'SKU': 'ID_SKU_VENTA', 'CD': 'DESCR_CENDIST'})

    for index, row in master.iterrows():

        if row['CANTIDAD HUB 3R'] != 0:
            dict_sku_cd[(row['ID_SKU_VENTA'], row['DESCR_CENDIST'])] = 1

        else:
            dict_sku_cd[(row['ID_SKU_VENTA'], row['DESCR_CENDIST'])] = 0
            
    assign_hub = (lambda x: x['DESCR_CENDIST'] if (x['ID_SKU_VENTA'], x['DESCR_CENDIST']) in \
                            dict_sku_cd and dict_sku_cd[(x['ID_SKU_VENTA'], x['DESCR_CENDIST'])] == 0 \
                            else hubs_dict[x['DESCR_CENDIST']])

else:
    assign_hub = lambda x: hubs_dict[x['DESCR_CENDIST']]
    
datos_para_agrupar = datos_limpios.copy()
datos_para_agrupar['DESCR_CENDIST'] = datos_para_agrupar.apply(assign_hub, axis=1)
datos_agrupados = datos_para_agrupar.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST', 'FECHA']).sum().reset_index()

In [16]:
stats_agrupados, porcentaje_ss_agrupado = stats(datos_agrupados, 1.28)

In [17]:
stats_agrupados

,ID_SKU_VENTA,DESCR_CENDIST,MEDIA,STD,COEF_VAR,SS,STOCK_TEORICO,PORCENTAJE_SS_DEL_TOTAL
0,515,ANTOFAGASTA,0.6201,1.0041,1.6193,1.2853,1.9054,0.6745
1,515,COPIAPO,0.2251,0.4634,2.0588,0.5931,0.8182,0.7249
2,515,COQUIMBO,0.9305,1.6295,1.7511,2.0857,3.0162,0.6915
3,515,COYHAIQUE,0.0473,0.1279,2.7034,0.1638,0.2111,0.7758
4,515,CURAUMA,0.1050,0.2715,2.5865,0.3475,0.4524,0.7680
...,...,...,...,...,...,...,...,...
13015,875225,VALDIVIA,0.0030,0.0202,6.7690,0.0259,0.0289,0.8965
13016,875359,SANTIAGO SUR,0.0084,0.2427,28.8444,0.3106,0.3190,0.9736
13017,875365,CURAUMA,0.0168,0.3430,20.3838,0.4390,0.4559,0.9631
13018,875365,SANTIAGO SUR,0.0084,0.2427,28.8444,0.3106,0.3190,0.9736


In [18]:
stats_agrupados[stats_agrupados['MEDIA'] != 0].to_csv('../stats_sku_entero/stats_agrupados_limpios_100%_90%conf.csv')

In [ ]:
demandas = stats_agrupados.groupby(by=['ID_SKU_VENTA']).sum().reset_index()[['ID_SKU_VENTA', 'MEDIA']]
# demandas

In [153]:
no_agrupado_agregado = stats_no_agrupados.groupby(by=['DESCR_CENDIST']).sum()[['MEDIA', 'SS', 'STOCK_TEORICO']]
no_agrupado_agregado = no_agrupado_agregado.reset_index()
# no_agrupado_agregado

In [154]:
agrupado_agregado = stats_agrupados.groupby(by=['DESCR_CENDIST']).sum()[['MEDIA', 'SS', 'STOCK_TEORICO']]
agrupado_agregado = agrupado_agregado.reset_index()
# agrupado_agregado

In [156]:
delta_pallets = cds_no_agrupados['STOCK_TEORICO'].sum() - cds_agrupados['STOCK_TEORICO'].sum()
delta_pallets

2843.9188887512064

In [ ]:
if restricted:
    diff = no_agrupado_agregado.merge(agrupado_agregado, on=['DESCR_CENDIST'], how='outer')
    diff = diff.fillna(0)
    diff['DELTA'] = diff['STOCK_TEORICO_y'] - diff['STOCK_TEORICO_x']
    deltaStocks = diff[['DESCR_CENDIST', 'DELTA']]
    
    OUTPUT_FILE_STOCKS = f'../data/Agrupación {porcentaje}%/inventario_final_agrupacion_{porcentaje}.csv'
    agrupado_agregado.to_csv(OUTPUT_FILE_STOCKS)
    
    OUTPUT_FILE_DELTAS = f'../data/Agrupación {porcentaje}%/deltas_inventario_agrupacion_{porcentaje}.csv'
    deltaStocks.to_csv(OUTPUT_FILE_DELTAS)

In [ ]:
if not restricted:
    hubs = list(set(hubs_dict.values()))

    diff = no_agrupado_agregado.merge(agrupado_agregado, how='outer', on=['DESCR_CENDIST'])
    diff = diff.fillna(0)
    diff['DELTA'] = diff['STOCK_TEORICO_y'] - diff['STOCK_TEORICO_x']

    diff['NEW_STOCK'] = 0

    diff['NEW_STOCK_CD'] = diff['MEDIA_x']
    diff['NEW_STOCK_HUB'] = diff['MEDIA_x'] + diff['SS_y']

    diff = diff[['DESCR_CENDIST', 'NEW_STOCK_HUB', 'NEW_STOCK_CD']]

    hubsStocks = diff[diff['DESCR_CENDIST'].isin(hubs)].reset_index()
    hubsStocks = hubsStocks[['DESCR_CENDIST', 'NEW_STOCK_HUB']]
    hubsStocks = hubsStocks.rename(columns={'NEW_STOCK_HUB': 'STOCK'})
    # hubsStocks

    cdStocks = diff[~diff['DESCR_CENDIST'].isin(hubs)].reset_index()
    cdStocks = cdStocks[['DESCR_CENDIST', 'NEW_STOCK_CD']]
    cdStocks = cdStocks.rename(columns={'NEW_STOCK_CD': 'STOCK'})
    # cdStocks

    newStocks = pd.concat([hubsStocks, cdStocks])
    newStocks = newStocks.sort_values(by=['DESCR_CENDIST'])
    # newStocks
    
    OUTPUT_FILE_STOCKS = '../data/Agrupacion total variabilidad/inventario_final_agrupacion_variabilidad_total.csv'
    newStocks.to_csv()

    oldStocks = no_agrupado_agregado[['DESCR_CENDIST', 'STOCK_TEORICO']]
    oldStocks = oldStocks.rename(columns={'STOCK_TEORICO': 'STOCK'})
    # oldStocks

    deltaStocks = oldStocks.merge(newStocks, on='DESCR_CENDIST')
    deltaStocks['DELTA'] = deltaStocks['STOCK_y'] - deltaStocks['STOCK_x'] 
    deltaStocks
    deltaStocks = deltaStocks[['DESCR_CENDIST', 'DELTA']]
    #deltaStocks

    OUTPUT_FILE_DELTAS = '../data/Agrupacion total variabilidad/deltas_inventario_agrupacion_variabilidad_total.csv'
    deltaStocks.to_csv(OUTPUT_FILE_DELTAS)

hubs = list(set(hubs_dict.values()))

diff = no_agrupado_agregado.merge(agrupado_agregado, how='outer', on=['DESCR_CENDIST'])
diff = diff.fillna(0)
diff['DELTA'] = diff['STOCK_TEORICO_y'] - diff['STOCK_TEORICO_x']

diff['NEW_STOCK'] = 0

diff['NEW_STOCK_CD'] = diff['MEDIA_x']
diff['NEW_STOCK_HUB'] = diff['MEDIA_x'] + diff['SS_y']

diff = diff[['DESCR_CENDIST', 'NEW_STOCK_HUB', 'NEW_STOCK_CD']]
# diff

hubsStocks = diff[diff['DESCR_CENDIST'].isin(hubs)].reset_index()
hubsStocks = hubsStocks[['DESCR_CENDIST', 'NEW_STOCK_HUB']]
hubsStocks = hubsStocks.rename(columns={'NEW_STOCK_HUB': 'STOCK'})
# hubsStocks

cdStocks = diff[~diff['DESCR_CENDIST'].isin(hubs)].reset_index()
cdStocks = cdStocks[['DESCR_CENDIST', 'NEW_STOCK_CD']]
cdStocks = cdStocks.rename(columns={'NEW_STOCK_CD': 'STOCK'})
# cdStocks

newStocks = pd.concat([hubsStocks, cdStocks])
newStocks = newStocks.sort_values(by=['DESCR_CENDIST'])
newStocks

oldStocks = no_agrupado_agregado[['DESCR_CENDIST', 'STOCK_TEORICO']]
oldStocks = oldStocks.rename(columns={'STOCK_TEORICO': 'STOCK'})
oldStocks

deltaStocks = oldStocks.merge(newStocks, on='DESCR_CENDIST')
deltaStocks['DELTA'] = deltaStocks['STOCK_y'] - deltaStocks['STOCK_x'] 
deltaStocks
deltaStocks = deltaStocks[['DESCR_CENDIST', 'DELTA']]

pd.set_option('display.float_format','{:.4f}'.format)
deltaStocks

OUTPUT_FILE_DELTAS = '../data/Agrupacion total variabilidad/deltas_inventario_agrupacion_variabilidad_total.csv'
deltaStocks.to_csv(OUTPUT_FILE_DELTAS)